## Exporting Data from GEE directly to Local Drive
## Use the following commands in miniconda prompt if geemap is not installed
##### conda create -n gee python
##### conda activate gee
##### conda install -c conda-forge geemap
##### conda install -c conda-forge spyder-kernels=2.4  -- in case spyder kernels are not installed

In [ ]:
import ee

In [1]:
ee.Authenticate() # If credentials already exist, they are automatically reused
# ee.Authenticate(force=True) # to create new credentials
ee.Initialize()


Successfully saved authorization token.


In [2]:
import geemap
Map = geemap.Map()

In [ ]:
Map

## Raw Javascript

In [ ]:
# // Define the region of interest (ROI) as a feature collection (e.g., from a table)
# //var roi = ee.FeatureCollection(table);
# //Map.addLayer(roi);
# // Define the latitude and longitude bounds for your area of interest
# var lat_min = 21.375;
# var lat_max = 32.125;
# var lon_min = 72.125;
# var lon_max = 89.875;

# // Create a rectangle geometry for the area of interest
# Map.addLayer(roi);
# // Print the area of interest
# print("Area of Interest:", roi);

# // Define the ImageCollection with the specified ID and filter it based on time and region.
# var collection = ee.ImageCollection("ECMWF/ERA5/DAILY")
#   .select(['maximum_2m_air_temperature',
#   // 'mean_2m_air_temperature',
#   'minimum_2m_air_temperature'
#   // 'total_precipitation',
#   // 'u_component_of_wind_10m',
#   // 'v_component_of_wind_10m'
#   ])
#   .filterBounds(roi)
#   .filterDate('1990-01-01', '1998-01-01')
#   ;

# // Create a function to extract values for all pixels in the ROI for each image
# var extractValues = function (image) {
#   var date = image.date().format('yyyy-MM-dd');

#   // Extract values for all pixels in the ROI
#   var values = image.sample({
#   region: roi,
#   scale: 27830,
#   geometries: true
# });
#   // Map over the values to create features
#   var features = values.map(function (feature) {
#     var properties = feature.toDictionary();

# // Get the geometry of the feature (pixel)
#     var geometry = feature.geometry();

#     // Get the latitude and longitude of the pixel center
#     var latLon = ee.List(geometry.coordinates().reverse());
#     var latitude = latLon.get(0);
#     var longitude = latLon.get(1);
 
   
#     // Set additional properties including latitude and longitude
#     properties = properties.combine(ee.Dictionary({
#       'date': date,
#       'latitude': latitude,
#       'longitude': longitude,
#       'maximum_2m_air_temperature': feature.getNumber('maximum_2m_air_temperature'),
#       // 'mean_2m_air_temperature': feature.getNumber('mean_2m_air_temperature'),
#       'minimum_2m_air_temperature': feature.getNumber('minimum_2m_air_temperature')
#       // 'total_precipitation': feature.getNumber('total_precipitation'),
#       // 'u_component_of_wind_10m': feature.getNumber('u_component_of_wind_10m'),
#       // 'v_component_of_wind_10m': feature.getNumber('v_component_of_wind_10m')
#     }));

#     return ee.Feature(null, properties);
#   });

#   return features;
# };

# // Map the extractValues function over the ImageCollection
# var extractedData = collection.map(extractValues).flatten();

# // Filter out features with null values
# extractedData = extractedData.filter(ee.Filter.notNull([
#   'latitude',
#   'longitude',
#   'maximum_2m_air_temperature',
#   // 'mean_2m_air_temperature',
#   'minimum_2m_air_temperature'
#   // 'total_precipitation',
#   // 'u_component_of_wind_10m',
#   // 'v_component_of_wind_10m'
# ]));

# // Print the first feature in the extractedData collection to inspect its properties
# print('Extracted Data:', extractedData.first());

# // Export the data to a CSV file.
# Export.table.toDrive({
#   collection: extractedData,
#   description: 'era5_data_1990_1997',
#   folder: 'GEE_exports', // Optional: specify a folder on Google Drive
#   fileNamePrefix: 'era5_data_1990_1997',
#   fileFormat: 'CSV'
# });

## Python API Script

In [ ]:
# Define the region of interest (ROI) as a feature collection (e.g., from a table)
roi = ee.FeatureCollection("projects/ee-rachitknl/assets/Rectangle")
# Define the latitude and longitude bounds for your area of interest
var lat_min = 21.375
var lat_max = 32.125
var lon_min = 72.125
var lon_max = 89.875

# Create a rectangle geometry for the area of interest
Map.addLayer(roi)
# Print the area of interest
print("Area of Interest:", roi)

# Define the ImageCollection with the specified ID and filter it based on time and region.
collection = ee.ImageCollection("ECMWF/ERA5/DAILY")
  .select(['maximum_2m_air_temperature',
  # 'mean_2m_air_temperature',
  'minimum_2m_air_temperature'
  # 'total_precipitation',
  # 'u_component_of_wind_10m',
  # 'v_component_of_wind_10m'
  ])
  .filterBounds(roi)
  .filterDate('1990-01-01', '1998-01-01')

# Create a function to extract values for all pixels in the ROI for each image
extractValues = function (image) {
date = image.date().format('yyyy-MM-dd')

# Extract values for all pixels in the ROI
values = image.sample({
region: roi,
scale: 27830,
geometries: true
})
# Map over the values to create features
features = values.map(function (feature) {
properties = feature.toDictionary()

# Get the geometry of the feature (pixel)
geometry = feature.geometry()

    # Get the latitude and longitude of the pixel center
    latLon = ee.List(geometry.coordinates().reverse())
    latitude = latLon.get(0)
    longitude = latLon.get(1)
 
   
    # Set additional properties including latitude and longitude
    properties = properties.combine(ee.Dictionary({
      'date': date,
      'latitude': latitude,
      'longitude': longitude,
      'maximum_2m_air_temperature': feature.getNumber('maximum_2m_air_temperature'),
      # // 'mean_2m_air_temperature': feature.getNumber('mean_2m_air_temperature'),
      'minimum_2m_air_temperature': feature.getNumber('minimum_2m_air_temperature')
      # // 'total_precipitation': feature.getNumber('total_precipitation'),
      # // 'u_component_of_wind_10m': feature.getNumber('u_component_of_wind_10m'),
      # // 'v_component_of_wind_10m': feature.getNumber('v_component_of_wind_10m')
    }));

    return ee.Feature(null, properties)
  })

  return features
}

# // Map the extractValues function over the ImageCollection
extractedData = collection.map(extractValues).flatten();

# // Filter out features with null values
extractedData = extractedData.filter(ee.Filter.notNull([
  'latitude',
  'longitude',
  'maximum_2m_air_temperature',
  # // 'mean_2m_air_temperature',
  'minimum_2m_air_temperature'
  # // 'total_precipitation',
  # // 'u_component_of_wind_10m',
  # // 'v_component_of_wind_10m'
]))

# // Print the first feature in the extractedData collection to inspect its properties
print('Extracted Data:', extractedData.first())

# // Export the data to a CSV file.
# Export.table.toDrive({
#   collection: extractedData,
#   description: 'era5_data_1990_1997',
#   folder: 'GEE_exports', // Optional: specify a folder on Google Drive
#   fileNamePrefix: 'era5_data_1990_1997',
#   fileFormat: 'CSV'
# });

In [ ]:
import os
os.chdir(r'E:\\Rachit\\IGBP_Project\\Data\\Weather_data\\ERA5\\Temp_2m')
geemap.ee_to_csv(extractedData, filename='era5_data_1990_1997.csv')

In [17]:
#  import geopandas as gpd
# Define regions of interest
# roi = gpd.read_file(r"\\172.16.20.21\wrd_p_igbp\IGBP_PROJECT_2023-24\IGBP_Project_files\Ganga_basin_shp\Ganga_Rac_shp.shp")
roi = ee.FeatureCollection("projects/ee-rachitknl/assets/Rectangle")
stations = ee.FeatureCollection("users/rachitknl/Ganga_stations")

def func_ctz(image):
  return image.clip(roi)
                  
selectedMonths = [1,2,3,4,5,6,7,8,9,10,11,12]

# Filter and select data
dataset = ee.ImageCollection('NASA/GDDP-CMIP6') \
                  .filter(ee.Filter.date('2021-01-01', '2026-01-01')) \
                  .filter(ee.Filter.eq('model', 'IITM-ESM')) \
                  .filter(ee.Filter.eq('scenario', 'ssp245')) \
                  .select('pr') \
                  .map(func_ctz)

# Precipitation
# pcp = dataset.select('pr')
filteredDataset = dataset.filter(ee.Filter.inList('month', selectedMonths))
# Visualization parameters
# pcp_vis = {
#   'min': 240,
#   'max': 310,
#   'palette': ['blue', 'purple', 'cyan', 'green', 'yellow', 'red'],
# }

# Create chart
# chart = ui.Chart.image.series(dataset, roi)
# print(chart)

# Center map and add layers
# Map.setCenter(77.06194261278883, 28.663690725304463, 7)
# Map.addLayer(roi)
# # Map.addLayer(stations)
# Map.addLayer(filteredDataset, pcp_vis, 'Precipitation(kg/m^2/s)')

# Stack bands and calculate means by region
stacked = filteredDataset.toBands()
means = stacked.reduceRegions(stations, ee.Reducer.mean().forEachBand(stacked))
# print(stacked.getInfo())

# Export data as CSV (commented out for illustration)
# Export.table.toDrive(means, 'weather_data', 'NEX-GDDP_GEE', '_', 'CSV')

# Looping through images for export (commented out for illustration)
# count = dataset.size()
# listOfImages = dataset.toList(count)

# for i in range(count.getInfo()):
#   image = ee.Image(listOfImages.get(i))
#   date = ee.Date(image.get('system:time_start')).format('YYYY-MM-DD')
#   filename = 'image_' + date
  
#   Export.image.toDrive({
#     'image': image,
#     'description': filename,
#     'folder': 'Your_Folder_Name',
#     # 'scale': 30,  # Adjust the scale according to the desired resolution
#     'region': roi
#   })


In [ ]:
Map

NameError: name 'Map' is not defined

In [18]:
import os
os.chdir(r'\\172.16.20.21\wrd_p_igbp\IGBP_PROJECT_2023-24\IGBP_Project_files\Data\Weather_data\CMIP6')

In [19]:
geemap.ee_to_csv(means, filename='ganga2021_2025.csv')